In [1]:
from datetime import datetime
import fnmatch
import pandas as pd

In [2]:
gatewayVolumesLoc = "./GatewayVolumes_2056.csv"
logisticChainFlowsLoc = "./LogisticChainFlows_2056.csv"

In [3]:
gatewayVolumes = pd.read_csv(gatewayVolumesLoc)
logisticChainFlows = pd.read_csv(logisticChainFlowsLoc)

In [4]:
gatewayMatchColNames = ['Gateway','Sector','Import / Export','Full / Empty']

In [5]:
yearColName = gatewayVolumes.columns[pd.Series(gatewayVolumes.columns).str.startswith('Y2')][0]

In [6]:
def matchStrings(primaryString, secondaryString):
    isMatch = False
    secondaryStringList = secondaryString.split(';')
    for i in range(len(secondaryStringList)):
        isMatch = isMatch | fnmatch.fnmatch(primaryString, secondaryStringList[i])
    return isMatch

def matchCols(primaryRow, secondaryRow, primaryColsList, secondaryColsList):
    rowMatch = True
    
    for j in range(len(primaryColsList)):
        rowMatch = rowMatch &\
                matchStrings(primaryRow[primaryColsList[j]],\
                             secondaryRow[secondaryColsList[j]])
    return rowMatch

In [7]:
from IPython.display import display

def pretty_print_df(df, title="DataFrame", max_rows=10):
    """Display a DataFrame with a title and limited rows"""
    print(f"\n{'='*50}\n{title}\n{'='*50}")
    display(df.head(max_rows))
    print(f"Shape: {df.shape}")
    if len(df) > max_rows:
        print(f"Showing first {max_rows} rows of {len(df)} total rows")

allChainFlowVolumes = pd.DataFrame()

for gatewayIt in range(len(gatewayVolumes)):
    gatewayRow = gatewayVolumes.iloc[gatewayIt]
    
    matchedFlows = logisticChainFlows[logisticChainFlows\
                                      .apply(lambda row:\
                                             matchCols(gatewayRow, row,\
                                                       gatewayMatchColNames,\
                                                       gatewayMatchColNames),\
                                             axis=1)].copy()

    pretty_print_df(matchedFlows, f"Matched Flows for Gateway: {gatewayRow['Gateway']}")
    if len(matchedFlows) == 0:
        print(f'INPUT DATA ERROR: No logistic chain flows corresponding to gateway {gatewayRow["Gateway"]}\n')
        continue
    
    matchedFlows['Input Link Share'] = 0
    matchedFlows['Output Link Share'] = 0
    
    matchedFlows[yearColName] = pd.to_numeric(matchedFlows[yearColName], errors='coerce')
    
    currentScale = matchedFlows.groupby(['From', 'From State'])\
                    .agg({yearColName: 'sum'})\
                    .reset_index().rename(columns={yearColName: 'Current Scale'})
    currentScale['Goal Scale'] = 1
    
    if len(currentScale.loc[abs(currentScale['Current Scale'] - 1) > 0.0000001]) > 0:
        print('INPUT DATA WARNING: Logistic chain flows that don’t add up to 100% - these have been scaled:\n')
        pretty_print_df(currentScale.loc[abs(currentScale['Current Scale'] - 1) > 0.0000001], 
                       "Scaled Flows (Current Scale ≠ 1)")

    matchedFlows = matchedFlows.merge(currentScale)
    matchedFlows[yearColName] = matchedFlows[yearColName] / matchedFlows['Current Scale'] * matchedFlows['Goal Scale']
    pretty_print_df(matchedFlows, f"Matched Flows After Scaling for Gateway: {gatewayRow['Gateway']}")
    
    if len(matchedFlows['Volume Entry'].unique()) > 1:
        print('INPUT DATA ERROR: Multiple values in Volume Entry column - only one entry point permitted per gateway\n')
        continue
        
    entryPoint = matchedFlows['Volume Entry'].iloc[0]
    entryState = matchedFlows[matchedFlows['From'] == entryPoint].iloc[0]['From State']
    toAdd = [
        pd.Series({
            'To': entryPoint,
            'Output Link Share': 1,
            'To State': entryState
        })
    ]
    
    initial_df = pd.DataFrame(toAdd)
    pretty_print_df(initial_df, f"Initial Link for Gateway: {gatewayRow['Gateway']}")

    iteration = 0
    while len(toAdd) > 0:
        iteration += 1
        currentLink = toAdd.pop(0)
        print(f"\nIteration {iteration}: Propagating from {currentLink['To']} ({currentLink['To State']}, {currentLink['Output Link Share']:.4f})")
        
        newInputVol = matchedFlows\
                .apply(lambda row: matchCols(currentLink, row,\
                                             ['To', 'To State'],\
                                             ['From', 'From State']),\
                       axis=1) * currentLink['Output Link Share']
        matchedFlows['Input Link Share'] = matchedFlows['Input Link Share'] + newInputVol
        
        newOutputVol = newInputVol * matchedFlows[yearColName]
        matchedFlows['Output Link Share'] = matchedFlows['Output Link Share'] + newOutputVol

        # Update new links to add
        toAddDF = matchedFlows[matchedFlows\
           .apply(lambda row: matchCols(currentLink, row,\
                                        ['To', 'To State'],\
                                        ['From', 'From State']),\
                  axis=1)]
        toAddVols = newOutputVol[newOutputVol > 0]
        
        # Pretty print the propagation
        if len(toAddDF) > 0:
            pretty_print_df(toAddDF[['From', 'To', 'From State', 'To State', yearColName, 'Input Link Share', 'Output Link Share']],
                          f"Propagated Flows from {currentLink['To']} ({currentLink['To State']})")
        
        for i in range(len(toAddDF)):
            toAdd.append(pd.Series({'To': toAddDF['To'].iloc[i], 'Output Link Share': toAddVols.iloc[i], 'To State': toAddDF['To State'].iloc[i]}))

    if abs(sum(matchedFlows[~matchedFlows['To State'].isin(set(matchedFlows['From State']))]['Output Link Share']) - 1) > 0.0000001:
        print('INPUT DATA WARNING: Final volumes should add up to 100%\n')
    
    matchedFlows = matchedFlows.filter(gatewayMatchColNames + ['Industry Class', 'Volume Entry', 'From', 'To', 'From State', 'To State', 'Run Type', 'Output Link Share'])

    allChainFlowVolumes = pd.concat([allChainFlowVolumes, matchedFlows], ignore_index=True)
    
    pretty_print_df(allChainFlowVolumes, f"Final Flow Shares for Gateway: {gatewayRow['Gateway']}")
#     break

# Pretty print the final combined result
pretty_print_df(allChainFlowVolumes, "All Chain Flow Volumes (Cumulative Result)")


Matched Flows for Gateway: Swanson Dock West


,Gateway,Sector,Import / Export,Full / Empty,Volume Entry,From,To,From State,To State,Run Type,Y2056
0,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,PRS Terminals WIFT / BIFT,Full,Full,RL,0.121442
1,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,PRS Terminals Others,Full,Full,RL,0.083671
2,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,Importers,Full,Empty,CR,0.176977
3,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,Transport Depots,Full,Full,BR,0.617909
4,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Empty,CR,0.700000
5,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Complete,IG,0.300000
6,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals Others,Importers,Full,Empty,CR,0.900000
7,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals Others,Importers,Full,Complete,IG,0.100000
8,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Transport Depots,Importers,Full,Empty,CR,1.000000
9,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Importers,PRS Terminals WIFT / BIFT,Empty,Empty,CR,0.036433


Shape: (17, 11)
Showing first 10 rows of 17 total rows

Matched Flows After Scaling for Gateway: Swanson Dock West


,Gateway,Sector,Import / Export,Full / Empty,Volume Entry,From,To,From State,To State,Run Type,Y2056,Input Link Share,Output Link Share,Current Scale,Goal Scale
0,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,PRS Terminals WIFT / BIFT,Full,Full,RL,0.121442,0,0,1.0,1
1,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,PRS Terminals Others,Full,Full,RL,0.083671,0,0,1.0,1
2,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,Importers,Full,Empty,CR,0.176977,0,0,1.0,1
3,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,Transport Depots,Full,Full,BR,0.617909,0,0,1.0,1
4,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Empty,CR,0.700000,0,0,1.0,1
5,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Complete,IG,0.300000,0,0,1.0,1
6,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals Others,Importers,Full,Empty,CR,0.900000,0,0,1.0,1
7,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals Others,Importers,Full,Complete,IG,0.100000,0,0,1.0,1
8,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Transport Depots,Importers,Full,Empty,CR,1.000000,0,0,1.0,1
9,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Importers,PRS Terminals WIFT / BIFT,Empty,Empty,CR,0.036433,0,0,1.0,1


Shape: (17, 15)
Showing first 10 rows of 17 total rows

Initial Link for Gateway: Swanson Dock West


...

Shape: (1, 7)

Iteration 11: Propagating from BIFT South Rail Terminal (Complete, 0.0139)

Final Flow Shares for Gateway: BIFT South Rail Terminal


,Gateway,Sector,Import / Export,Full / Empty,Volume Entry,From,To,From State,To State,Run Type,Output Link Share
0,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,PRS Terminals WIFT / BIFT,Full,Full,RL,0.121442
1,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,PRS Terminals Others,Full,Full,RL,0.083671
2,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,Importers,Full,Empty,CR,0.176977
3,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,Transport Depots,Full,Full,BR,0.617909
4,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Empty,CR,0.085009
5,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Complete,IG,0.036433
6,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals Others,Importers,Full,Empty,CR,0.075304
7,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals Others,Importers,Full,Complete,IG,0.008367
8,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Transport Depots,Importers,Full,Empty,CR,0.617909
9,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Importers,PRS Terminals WIFT / BIFT,Empty,Empty,CR,0.034800


Shape: (238, 11)
Showing first 10 rows of 238 total rows

All Chain Flow Volumes (Cumulative Result)


,Gateway,Sector,Import / Export,Full / Empty,Volume Entry,From,To,From State,To State,Run Type,Output Link Share
0,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,PRS Terminals WIFT / BIFT,Full,Full,RL,0.121442
1,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,PRS Terminals Others,Full,Full,RL,0.083671
2,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,Importers,Full,Empty,CR,0.176977
3,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Swanson Dock West,Transport Depots,Full,Full,BR,0.617909
4,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Empty,CR,0.085009
5,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Complete,IG,0.036433
6,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals Others,Importers,Full,Empty,CR,0.075304
7,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,PRS Terminals Others,Importers,Full,Complete,IG,0.008367
8,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Transport Depots,Importers,Full,Empty,CR,0.617909
9,Swanson Dock West,I&MC,Import,Full,Swanson Dock West,Importers,PRS Terminals WIFT / BIFT,Empty,Empty,CR,0.034800


Shape: (238, 11)
Showing first 10 rows of 238 total rows


In [9]:
allChainFlowVolumes.to_csv("allChainFlowVolumes.csv")